<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Script-setup" data-toc-modified-id="Script-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script setup</a></span></li><li><span><a href="#Data-directory-preparention" data-toc-modified-id="Data-directory-preparention-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data directory preparention</a></span></li><li><span><a href="#Data-file-preperation" data-toc-modified-id="Data-file-preperation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data file preperation</a></span><ul class="toc-item"><li><span><a href="#input-directory-(all-general-raw-data-files)" data-toc-modified-id="input-directory-(all-general-raw-data-files)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><code>input</code> directory (all general raw data files)</a></span></li><li><span><a href="#processed-directory-(all-data-files-produced-in-this-repository)" data-toc-modified-id="processed-directory-(all-data-files-produced-in-this-repository)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span><code>processed</code> directory (all data files produced in this repository)</a></span></li></ul></li><li><span><a href="#Load-data-functions" data-toc-modified-id="Load-data-functions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load data functions</a></span></li><li><span><a href="#Load-data-sets" data-toc-modified-id="Load-data-sets-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Load data sets</a></span></li><li><span><a href="#ENTSO-E-generation-per-technology-CI" data-toc-modified-id="ENTSO-E-generation-per-technology-CI-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>ENTSO-E generation per technology CI</a></span><ul class="toc-item"><li><span><a href="#EF-preparation" data-toc-modified-id="EF-preparation-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>EF preparation</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Plots</a></span></li></ul></li><li><span><a href="#ENTSO-E-Power-Stats-generation-per-technology-CO2-Signal" data-toc-modified-id="ENTSO-E-Power-Stats-generation-per-technology-CO2-Signal-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>ENTSO-E Power Stats generation per technology CO2 Signal</a></span></li><li><span><a href="#Compair-Bottom-Up-and-Top-down" data-toc-modified-id="Compair-Bottom-Up-and-Top-down-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Compair Bottom Up and Top down</a></span></li><li><span><a href="#DK-wärme-wird-dort-viel-von-Kraftwerken-genutzt" data-toc-modified-id="DK-wärme-wird-dort-viel-von-Kraftwerken-genutzt-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>DK wärme wird dort viel von Kraftwerken genutzt</a></span></li><li><span><a href="#EE-hat-ganz-viel-other-fossil-(wir-haben-keine-EF-ausgerechnet)" data-toc-modified-id="EE-hat-ganz-viel-other-fossil-(wir-haben-keine-EF-ausgerechnet)-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>EE hat ganz viel other fossil (wir haben keine EF ausgerechnet)</a></span></li><li><span><a href="#LT" data-toc-modified-id="LT-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>LT</a></span></li><li><span><a href="#LV" data-toc-modified-id="LV-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>LV</a></span></li><li><span><a href="#NO" data-toc-modified-id="NO-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>NO</a></span></li><li><span><a href="#Plotting-CI" data-toc-modified-id="Plotting-CI-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Plotting CI</a></span></li><li><span><a href="#Export-final-CO2-signals" data-toc-modified-id="Export-final-CO2-signals-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Export final CO2 signals</a></span></li><li><span><a href="#Plottting" data-toc-modified-id="Plottting-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Plottting</a></span></li><li><span><a href="#Create-a-dataframe" data-toc-modified-id="Create-a-dataframe-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Create a dataframe</a></span></li><li><span><a href="#Reorder-it-following-the-values-of-the-first-value:" data-toc-modified-id="Reorder-it-following-the-values-of-the-first-value:-19"><span class="toc-item-num">19&nbsp;&nbsp;</span>Reorder it following the values of the first value:</a></span></li><li><span><a href="#The-vertical-plot-is-made-using-the-hline-function" data-toc-modified-id="The-vertical-plot-is-made-using-the-hline-function-20"><span class="toc-item-num">20&nbsp;&nbsp;</span>The vertical plot is made using the hline function</a></span></li><li><span><a href="#I-load-the-seaborn-library-only-to-benefit-the-nice-looking-feature" data-toc-modified-id="I-load-the-seaborn-library-only-to-benefit-the-nice-looking-feature-21"><span class="toc-item-num">21&nbsp;&nbsp;</span>I load the seaborn library only to benefit the nice looking feature</a></span></li><li><span><a href="#Add-title-and-axis-names" data-toc-modified-id="Add-title-and-axis-names-22"><span class="toc-item-num">22&nbsp;&nbsp;</span>Add title and axis names</a></span></li></ul></div>

# Introduction

This notebook uses the calculated EFs to calculate the CI for EU countries.

# Script setup

In [37]:
import os
import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from IPython.display import Image 
import datetime
from matplotlib.lines import Line2D

#GEO
import geopandas as gpd
#from pyproj import CRS
#from pyproj import Transformer
#from shapely.geometry import Point

%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_theme()

#helpers
from helpers import change_ENTSOE_ProductionTypeName

# Data directory preparention

Create input, processed and output folders if they don't exist
If the paths are relative, the corresponding folders will be created inside the current working directory.
- input -> all needed input data
- processed -> save point and exchange with other scripts
- output -> final emission factors

In [38]:
input_directory_path = os.path.join('input')
processed_directory_path = 'processed'
output_directory_path = os.path.join('output')

os.makedirs(input_directory_path, exist_ok=True)
os.makedirs(processed_directory_path, exist_ok=True)
os.makedirs(output_directory_path, exist_ok=True)

# Data file preperation

## `input` directory (all general raw data files)

- 1) ENTSO-E production per type
Production per type from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Production Type Available online: 
https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show
Processed with the following script that converts the data to hourly resolution (private script from INATECH):


 File - > time_series_60min_multiindex.csv - > generation per technology
 
- 2) ENTSO-E production per unit
Production per Unit from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Generation Unit Available online: https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show (accessed on Apr 29, 2020).
Processed with the following script that converts the data to hourly resolution (private script from INATECH):
https://github.com/INATECH-CIG/entso-e_GenerationOutputPerUnit

File - > gen_data.csv -> hourly generation data per unit

In [39]:
# Checks if the the input directories are empty or not
# Checks all filenames in the input directory

if not os.listdir(input_directory_path) :
    print("The directory for the bootom up method is empty. Please provide the data to the directory as described in the instructions above.")


filenames = [os.path.join(input_directory_path, fn) for fn in os.listdir(input_directory_path)]

print(filenames)

['input\\Bootom_up_methode', 'input\\CO2_Signal', 'input\\ember_EF.csv', 'input\\entso.csv', 'input\\entsoe-statistical-factsheet-2018-stacked.csv', 'input\\entsoe_gen_type_hourly_table.csv', 'input\\entsoe_gen_type_hourly_table1.csv', 'input\\entso_gen_type_hourly_table_scaled.csv', 'input\\entso_scaled.csv', 'input\\gen_data_2015.csv', 'input\\gen_data_2016.csv', 'input\\gen_data_2017.csv', 'input\\gen_data_2018.csv', 'input\\Matching', 'input\\MDV_table.csv', 'input\\time_series_60min_multiindex.csv', 'input\\top_down_methode', 'input\\unit_data_2015.csv', 'input\\unit_data_2016.csv', 'input\\unit_data_2017.csv', 'input\\unit_data_2018.csv']


## `processed` directory (all data files produced in this repository)

- 1) EF per type and county

File - > EF_bottom_up_methode.csv

- 2) EF per unit

File -> generation_unit_EF_bottom_up_methode.csv

In [40]:
# Checks if the the input directories are empty or not
# Checks all filenames in the input directory

if not os.listdir(processed_directory_path) :
    print("The directory for the CO2 Signal is empty. Please provide the data to the directory as described in the instructions above.")


filenames = [os.path.join(processed_directory_path, fn) for fn in os.listdir(processed_directory_path)]

print(filenames)

['processed\\CI_0_top_down.csv', 'processed\\CI_1_top_down.csv', 'processed\\EF_bottom_up_methode.csv', 'processed\\generation_unit_EF_bottom_up_methode.csv']


# Load data functions

In [41]:
def load_generation_per_unit(path, fn):
    """
    Load the ENTSO-E gernation per unit data.
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    generation = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0,parse_dates=True)
    
    return generation

def load_unit_CO2_data(path, fn):
    """
    Own generation unit information with CO2 information per unit
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0)
    
    # set name for the index
    #df.index.set_names('eic_g', inplace=True)

    return df

def load_EF(path, fn):
    """
    Own generation EF per tech and country
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=[0,1])
    
    return df

def load_timeseries_ENTSOE(path, fn):
    """
    Read generation data from time-series package own modification.


    Parameters
    ----------
    path : TYPE
        DESCRIPTION.
    fn : TYPE
        DESCRIPTION.

    Returns
    -------
    generation : TYPE
        DESCRIPTION.

    """
   
    generation = (pd.read_csv(os.path.join(path, fn), index_col=[0], header=[0, 1], parse_dates=True)
                    .dropna(how="all", axis=0))
       
    
    return generation

# Load data sets

#### Load ENTSO-E generation per type

In [42]:
entsoe_gen_type = load_timeseries_ENTSOE(input_directory_path, 'entso_gen_type_hourly_table_scaled.csv')

#### Load EF per technology

In [43]:
EF = load_EF(processed_directory_path, 'EF_bottom_up_methode.csv')

In [44]:
EF.head(10)

EF Verified_EF  EF_round_in_kwh
MapCode ProductionTypeName                                       
AT      gas                 0.288711        True           288.71
        hard_coal           0.884070        True           884.07
BE      gas                 0.389641        True           389.64
CZ      hard_coal           0.985546        True           985.55
        lignite             0.928296        True           928.30
DE      gas                 0.334091        True           334.09
        hard_coal           0.871051        True           871.05
        lignite             1.125560        True          1125.56
        other_fossil        1.619695        True          1619.69
DK      gas                 0.329783       False           329.78

# ENTSO-E generation per technology CI

In this section we using the country specific EF for generating the CI for all EU countries. The production data are based on the production per type data from ENTSO-E

In [45]:
# take a look at the generation data
entsoe_gen_type['GB'].sum()/1000

variable
biomass           17900.0
gas              129900.0
hard_coal         16800.0
hydro              5900.0
nuclear           60700.0
other_fossil          0.0
solar             11800.0
wind_offshore     16700.0
wind_onshore      25600.0
dtype: float64

## EF preparation 

In [46]:
# Mean EF for all calculated countries and technologies

EF_mean = EF.groupby('ProductionTypeName').mean()
EF_mean.loc['other_fossil'] = 0.75
EF_mean = EF_mean.EF.to_dict()


print(EF_mean)

{'gas': 0.3665166624413151, 'hard_coal': 0.9088482292089693, 'lignite': 1.180877690651492, 'other_fossil': 0.75}


In [47]:
# If no country-specific EFs are available and also no mean EF is available for a country and technology, we use the following EFs. Source:

EF_default = {'biomass':0,
            'geothermal':0,
            'hydro':0,
            'nuclear':0,
            'oil':0.8,
            'other_renewable':0,
            'solar':0,
            'waste':0.5,
            'wind_offshore':0,
            'wind_onshore':0,
            'marine':0,
            'oil shale':0.51,
            'other_fossil': 0.75}


In [48]:
# Create a dataframe with the EF for each country and technology. If no specific EF is available, we use the default EF .
EF_multi = {}

for i in entsoe_gen_type.columns:
    if i in EF:
        EF_multi[i] = EF[i]
    elif i[1] in EF_mean:
        EF_multi[i] = EF_mean[i[1]]
    else:
        EF_multi[i] = EF_default[i[1]]

In [49]:
# calculate emissions based on EF and generation per type
CO2_emission_entsoe_gen_type = entsoe_gen_type * EF_multi

In [50]:
# From the previously calculated emissions, an emission signal (g CO2 / kWh) can now be calculated for each country.
# Since emissions and generation are available per technology, they must first be grouped and summed. 

# Emission in g CO2
CO2_emission = (CO2_emission_entsoe_gen_type * 1000000).groupby(level=0, axis=1).sum()
# generation in kWh
Generation = (entsoe_gen_type * 1000).groupby(level=0, axis=1).sum()

# CI in g CO2/kWh
Carbon_intensitiy_entsoe_gen_type = CO2_emission.div(Generation)

In [51]:
Carbon_intensitiy_entsoe_gen_type.head(5)

region,AT,BE,BG,CH,CY,CZ,DE,DE_50hertz,DE_LU,DE_amprion,...,PT,RO,RS,SE,SE_1,SE_2,SE_3,SE_4,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,96.245777,86.511121,445.827602,0.0,714.497120,483.489191,207.906968,286.123166,NaN,348.405201,...,133.707935,374.099352,879.225413,17.171539,0.0,0.0,0.0,0.0,274.681414,156.346201
2018-01-01 01:00:00+00:00,95.775234,90.920079,446.001878,0.0,706.883241,480.252606,206.531643,283.818376,NaN,346.439320,...,138.154869,364.935210,902.858444,17.343350,0.0,0.0,0.0,0.0,269.809900,155.128339
2018-01-01 02:00:00+00:00,102.458141,95.068925,446.860983,0.0,669.643451,476.815314,203.284450,286.848688,NaN,347.683842,...,129.603411,383.590409,909.342872,16.128131,0.0,0.0,0.0,0.0,250.278226,149.611504
2018-01-01 03:00:00+00:00,107.860447,94.498077,446.377176,0.0,668.807796,479.751242,203.985534,283.012426,NaN,350.168792,...,134.892045,393.300909,890.098963,16.175121,0.0,0.0,0.0,0.0,257.859306,140.088761
2018-01-01 04:00:00+00:00,106.579828,94.911063,447.662192,0.0,701.969841,485.979643,202.091847,282.316768,NaN,347.849836,...,135.347004,397.361279,906.729967,16.007878,0.0,0.0,0.0,0.0,259.976557,142.953895


In [52]:
mean_Carbon_intensitiy_entsoe_gen_type = CO2_emission.sum() / Generation.sum()

In [53]:
Carbon_intensitiy_entsoe_gen_type_mean = Carbon_intensitiy_entsoe_gen_type.copy()


In [54]:
for i in Carbon_intensitiy_entsoe_gen_type_mean.columns:
    Carbon_intensitiy_entsoe_gen_type_mean[i] = mean_Carbon_intensitiy_entsoe_gen_type[i]

In [55]:
Carbon_intensitiy_entsoe_gen_type_mean.head()

region,AT,BE,BG,CH,CY,CZ,DE,DE_50hertz,DE_LU,DE_amprion,...,PT,RO,RS,SE,SE_1,SE_2,SE_3,SE_4,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,131.671626,156.848575,487.870071,0.0,720.0,564.572184,444.58543,628.510205,434.411358,621.298896,...,282.446654,325.997145,841.923201,13.356164,0.0,0.0,0.0,0.0,329.900718,158.419555
2018-01-01 01:00:00+00:00,131.671626,156.848575,487.870071,0.0,720.0,564.572184,444.58543,628.510205,434.411358,621.298896,...,282.446654,325.997145,841.923201,13.356164,0.0,0.0,0.0,0.0,329.900718,158.419555
2018-01-01 02:00:00+00:00,131.671626,156.848575,487.870071,0.0,720.0,564.572184,444.58543,628.510205,434.411358,621.298896,...,282.446654,325.997145,841.923201,13.356164,0.0,0.0,0.0,0.0,329.900718,158.419555
2018-01-01 03:00:00+00:00,131.671626,156.848575,487.870071,0.0,720.0,564.572184,444.58543,628.510205,434.411358,621.298896,...,282.446654,325.997145,841.923201,13.356164,0.0,0.0,0.0,0.0,329.900718,158.419555
2018-01-01 04:00:00+00:00,131.671626,156.848575,487.870071,0.0,720.0,564.572184,444.58543,628.510205,434.411358,621.298896,...,282.446654,325.997145,841.923201,13.356164,0.0,0.0,0.0,0.0,329.900718,158.419555


## Plots

# ENTSO-E Power Stats generation per technology CO2 Signal

In [61]:
MDV = pd.read_csv(input_directory_path + '/MDV_table.csv', index_col=[0], header=[0, 1], parse_dates=True)

In [62]:
MDV.loc['2018'].head(5)

Country                 AL                                             \
ProductionTypeName biomass  gas hard_coal       hydro lignite nuclear   
index                                                                   
2018-01-01             0.0  0.0       0.0  274.666667     0.0     0.0   
2018-02-01             0.0  0.0       0.0  331.000000     0.0     0.0   
2018-03-01             0.0  0.0       0.0  406.000000     0.0     0.0   
2018-04-01             0.0  0.0       0.0  372.666667     0.0     0.0   
2018-05-01             0.0  0.0       0.0  317.666667     0.0     0.0   

Country                                                      ...        TR  \
ProductionTypeName other_fossil other_renewable solar waste  ... hard_coal   
index                                                        ...             
2018-01-01                  0.0             0.0   0.0   0.0  ...   6112.94   
2018-02-01                  0.0             0.0   0.0   0.0  ...   5643.05   
2018-03-01                  0.0             0.0   0.0   0.0  ...   4826.60   
2018-04-01                  0.0             0.0   0.0   0.0  ...   4138.59   
2018-05-01                  0.0             0.0   0.0   0.0  ...   5732.74   

Country                                                                        \
ProductionTypeName        hydro  lignite nuclear other_fossil other_renewable   
index                                                                           
2018-01-01          1508.030000  3611.22     0.0       25.668      199.326667   
2018-02-01          1195.396667  3511.82     0.0       20.726      175.790000   
2018-03-01          2084.640000  3781.64     0.0       19.966      193.436667   
2018-04-01          2109.743333  3629.94     0.0       22.210      184.626667   
2018-05-01          2160.496667  3790.46     0.0       26.972      180.926667   

Country                                                      
ProductionTypeName   solar waste wind_offshore wind_onshore  
index                                                        
2018-01-01          241.15   0.0           0.0      1811.56  
2018-02-01          349.37   0.0           0.0      1673.63  
2018-03-01          381.19   0.0           0.0      2045.82  
2018-04-01          710.82   0.0           0.0      1164.12  
2018-05-01          730.28   0.0           0.0      1217.68  

[5 rows x 432 columns]

In [63]:
# convert to MWh
MDV = MDV * 1000

In [64]:
# Create a dataframe with the EF for each country and technology. If no specific EF is available, we use the default EF .
EF_multi = {}

for i in MDV.columns:
    if i in EF:
        EF_multi[i] = EF[i]
    elif i[1] in EF_mean:
        EF_multi[i] = EF_mean[i[1]]
    else:
        EF_multi[i] = EF_default[i[1]]

In [65]:
# calculate emissions based on EF and generation per type
CO2_emission_MDV = MDV * EF_multi

In [66]:
# From the previously calculated emissions, an emission signal (g CO2 / kWh) can now be calculated for each country.
# Since emissions and generation are available per technology, they must first be grouped and summed. 
CO2_emission_MDV = (CO2_emission_MDV * 1000000).groupby(level=0, axis=1).sum()
Generation_MDV = (MDV * 1000).groupby(level=0, axis=1).sum()

Carbon_intensitiy_MDV = CO2_emission_MDV.div(Generation_MDV)

In [67]:
#take only year 2018
Carbon_intensitiy_MDV = Carbon_intensitiy_MDV.loc['2018']

In [68]:


mean_Carbon_intensitiy_MDV = CO2_emission_MDV.loc['2018'].sum() / Generation_MDV.loc['2018'].sum()

In [69]:
MDV['DE'].sum().sum()

2090735376.6666665

# Compair Bottom Up and Top down

In [70]:
CI_1_top_down = pd.read_csv(os.path.join(processed_directory_path,'CI_1_top_down.csv'), index_col=0)
CI_0_top_down = pd.read_csv(os.path.join(processed_directory_path,'CI_0_top_down.csv'), index_col=0)


#CI_top_down.rename(columns={'UK':'GB'}, inplace = True)

In [71]:
CI_compar = pd.DataFrame()
CI_compar.index.rename('country',inplace=True)

In [72]:
CI_compar.index

Index([], dtype='object', name='country')

In [73]:
CI_compar['CI_1_top_down'] = CI_1_top_down.loc[2018].filter(mean_Carbon_intensitiy_entsoe_gen_type.index.to_list())

In [74]:
CI_compar['CI_0_top_down'] = CI_0_top_down.loc[2018].filter(mean_Carbon_intensitiy_entsoe_gen_type.index.to_list())

In [78]:
CI_compar['CI_bottom_up'] = mean_Carbon_intensitiy_entsoe_gen_type.filter(CI_compar.index.to_list())

CI_compar['diff_bottom_up_vs_CI_0_top_down'] =CI_compar['CI_bottom_up'] -  CI_compar['CI_0_top_down']
CI_compar['diff_pro_0'] = round((100 / CI_compar['CI_bottom_up']) * CI_compar['diff_bottom_up_vs_CI_0_top_down'])

CI_compar['diff_bottom_up_vs_CI_1_top_down'] =CI_compar['CI_bottom_up'] -  CI_compar['CI_1_top_down']
CI_compar['diff_pro_1'] = round((100 / CI_compar['CI_bottom_up']) * CI_compar['diff_bottom_up_vs_CI_1_top_down'])

In [80]:
round(CI_compar, 2).sort_values(by='diff_pro_1')

,CI_1_top_down,CI_0_top_down,CI_bottom_up,diff_bottom_up_vs_CI_0_top_down,diff_pro_0,diff_bottom_up_vs_CI_1_top_down,diff_pro_1
country,,,,,,,
NO,18.28,22.38,8.11,-14.27,-176.0,-10.17,-125.0
FR,53.18,62.31,36.63,-25.68,-70.0,-16.55,-45.0
EE,886.49,932.32,647.06,-285.27,-44.0,-239.43,-37.0
BE,205.45,231.59,156.85,-74.75,-48.0,-48.60,-31.0
NL,437.55,500.84,381.43,-119.41,-31.0,-56.12,-15.0
GB,241.17,241.17,220.40,-20.77,-9.0,-20.77,-9.0
GR,649.70,654.23,597.45,-56.78,-10.0,-52.25,-9.0
PT,306.45,337.04,282.45,-54.60,-19.0,-24.00,-8.0
IE,347.35,347.35,322.48,-24.87,-8.0,-24.87,-8.0


In [36]:
round(CI_compar, 2).to_csv(output_directory_path + '/CI_table.csv')




CI_compar['CI_MDV'] = mean_Carbon_intensitiy_MDV
CI_compar['CI_FACT'] = mean_Carbon_intensitiy_FACT

CI_compar['diff_MDV'] =CI_compar['CI_MDV'] -  CI_compar['CI_top_down_ref']
CI_compar['diff_FACT'] =CI_compar['CI_FACT'] -  CI_compar['CI_top_down_ref']


CI_compar['diff_MDV_pro'] = round((100 / CI_compar['CI_top_down_ref']) * CI_compar['diff_MDV'])

CI_compar['diff_FACT_pro'] = round((100 / CI_compar['CI_top_down_ref']) * CI_compar['diff_FACT'])


entsoe_gen_type['SE'].sum()/1000

mean_Carbon_intensitiy_entsoe_gen_type['SE'].sum()/1000

CI_compar
#gründe warum CI_bottom up zu hoch
# DK wärme wird dort viel von Kraftwerken genutzt
# EE hat ganz viel other fossil (wir haben keine EF ausgerechnet)
# LT
# LV
# NO

Information about the data shown. The reference value shown here is not an official value, but can be used as a reference, since it is based on a documented and freely available method. 

energy_comp = pd.DataFrame()

energy_comp['MDV'] = MDV['2018'].sum()

energy_comp['entsoe'] = entsoe_gen_type.sum()

energy_comp['diff'] = energy_comp['MDV'] - energy_comp['entsoe']
energy_comp['missing_pro'] = round((100 / energy_comp['MDV']) * energy_comp['diff'])

energy_comp.loc['DE']

ember_CF = pd.read_csv(os.path.join(input_directory_path,'ember_EF.csv'))

# Plotting CI

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world[world.name == 'Germany']

import pycountry



pycountry.countries.search_fuzzy('Norway')

import numpy as np


def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_2
    except:
        return np.nan

world["country_code"] = world["name"].apply(lambda country: do_fuzzy_search(country))

world.set_index('country_code', inplace=True)

world['CI'] = Carbon_intensitiy_entsoe_gen_type.mean()

world[world.name == 'France']

world.plot(column='CI')

from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1)

fig.suptitle("Density difference plot for " + country, fontsize=16)

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=0.1)

world.plot(column='CI',
           ax=ax,
           cax=cax,
           legend=True,
           legend_kwds={'label': "Capacity density diff per NUTS region [MW/km²]", 'orientation': "vertical"},
           cmap='Blues',
           edgecolor='k')


ax.text(0.05, 0.05,'Mean Cap. difference ' + str(round(world.CI.mean())) + ' [MW]',\
        ha='left', va='center', transform=ax.transAxes, fontsize=15, color='white',\
        bbox={'facecolor': 'royalblue', 'alpha': 0.5, 'pad': 5})

ax.title.set_text('Density difference plot for capacities in the NUTS regions')


# Export final CO2 signals

Carbon_intensitiy_entsoe_gen_type.to_csv(processed_directory_path + '/CI_bottom_up_methode.csv')
Carbon_intensitiy_entsoe_gen_type.to_csv(output_directory_path + '/CI_bottom_up_methode.csv')

# Plottting 

CI_compar.index

import numpy as np
# Create a dataframe
#value1=np.random.uniform(size=20)
#value2=value1+np.random.uniform(size=20)/4
#df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'value1':value1 , 'value2':value2 })

CI_compar

# Reorder it following the values of the first value:
ordered_df = CI_compar.sort_values(by='CI_bottom_up')
my_range=range(1,len(CI_compar.index)+1)
 
# The vertical plot is made using the hline function
# I load the seaborn library only to benefit the nice looking feature
import seaborn as sns
plt.hlines(y=my_range, xmin=ordered_df['CI_top_down_ref'], xmax=ordered_df['CI_bottom_up'], color='grey', alpha=0.4)
plt.scatter(ordered_df['CI_top_down_ref'], my_range, color='skyblue', alpha=1, label='CI_top_down_ref')
plt.scatter(ordered_df['CI_bottom_up'], my_range, color='green', alpha=0.4 , label='CI_bottom_up')
plt.legend()
 
# Add title and axis names
plt.yticks(my_range, ordered_df.index)
plt.title("Comparison of the CI_top_down_ref and the CI_bottom_up", loc='left')
plt.xlabel('Value of the variables')
#plt.ylabel(ordered_df.index)

df